In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 23
TITLE = "21 February 2021"
SUBTITLE = "8 AM 22 February 2021"
sts = utc(2021, 2, 21, 18)
ets = utc(2021, 2, 22, 17, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [16]:
cull = [407, 'DUBI4', 61, 58, 54]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
    [87, 6.],
    [340, 2],
    [189, 5],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=6, upper=10)
res = mp.postprocess(filename='210222.png')
mp.close()

    state  wfo  val         lon        lat                               geo  \
1      IA  OAX  9.5  -96.100000  42.030000    POINT (322077.683 -322558.334)   
2      IA  OAX  9.0  -96.060000  41.860000    POINT (326259.835 -341281.677)   
3      IA  OAX  9.0  -95.790000  41.560000    POINT (350251.714 -373486.502)   
4      IA  OAX  8.5  -95.860000  41.430000    POINT (345138.428 -388218.019)   
5      NE  OAX  8.5  -95.920000  41.180000    POINT (341472.631 -416228.520)   
6      WI  ARX  8.5  -90.680000  43.740000     POINT (746744.747 -97298.834)   
7      IA  OAX  8.5  -95.900000  41.590000    POINT (340948.673 -370617.422)   
8      IA  OAX  8.5  -95.780000  41.590000    POINT (350916.925 -370112.560)   
9      IA  ARX  8.3  -92.550000  43.360000    POINT (601311.670 -154869.369)   
10     IA  OAX  8.3  -95.320000  41.660000    POINT (388690.311 -360270.925)   
11     NE  OAX  8.2  -95.970000  41.340000    POINT (336451.167 -398667.535)   
12     WI  ARX  8.2  -90.270000  43.9200